# Install and import requirements

In [1]:
!pip install openai

# Upload unlabelled train data

*  unlabelled train data - 'mushroom.en-train_nolabel.v1.json'

In [2]:
from google.colab import files
up = files.upload()

Saving mushroom.en-train_nolabel.v1.jsonl to mushroom.en-train_nolabel.v1.jsonl


In [3]:
import json

def load_jsonl_file_to_records(filename):
    records = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                records.append(json.loads(line))

        return pd.DataFrame(records)
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None

In [4]:
import pandas as pd
train_df = load_jsonl_file_to_records('mushroom.en-train_nolabel.v1.jsonl')

In [5]:
print("Length of unlabelled train file:", len(train_df))

Length of unabelled train file: 809


# Label the data

1) Generate Wikipedia Summary: Generate a summary from Wikipedia based on self-checking GPT API.

2) Correct LLM Output: Correct the LLM's output based on the Wikipedia summary using GPT API.

In [7]:
import pandas as pd
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(
    api_key=""
)

def generate_summary_with_self_check(model_input, num_responses=3):
    """
    Generate a Wikipedia-based summary and self-check it for consistency.
    """
    prompt = (
        f"Generate a concise Wikipedia-based summary for the following question:\n"
        f"'{model_input}'\n"
        f"Ensure the summary contains only factual and relevant information."
    )
    responses = []

    # Generate multiple responses for self-checking
    for _ in range(num_responses):
        try:
            print('Sending request in "generate_summary_with_self_check"')
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are an expert summarizer who generates accurate and concise summaries."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=500,
                temperature=0.0
            )
            responses.append(response.choices[0].message.content.strip())
        except Exception as e:
            responses.append(f"Error generating summary: {e}")

    # Compare responses for consistency
    return self_check_responses(responses)

def self_check_responses(responses):
    """
    Perform self-checking for multiple generated summaries.
    """
    prompt = (
        f"Here are multiple summaries generated for the same query:\n\n"
        f"{chr(10).join([f'{i+1}. {response}' for i, response in enumerate(responses)])}\n\n"
        f"Please:\n"
        f"Please provide the most accurate and consistent summary only."
    )
    try:
        print('Sending request in "self_check_responses"')
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert at verifying factual consistency in summaries, and return the best summary"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error during self-checking: {e}"

def generate_corrected_output(summary, model_output):
    """Generate a corrected version of the model output using the Wikipedia-based summary."""
    prompt = (
        f"The following text is a Wikipedia-based summary:\n"
        f"{summary}\n\n"
        f"The model generated the following output:\n"
        f"{model_output}\n\n"
        f"Correct the model's output using the Wikipedia-based summary. "
        f"Do not change the grammatical structure of the original text, but replace hallucinated or overgenerated words "
        f"with accurate ones from the summary. Provide only the corrected output."
    )
    try:
        print('Sending request in "generate_corrected_output"')
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert editor who ensures factual accuracy."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating corrected output: {e}"


if __name__ == "__main__":
    # Sample training and validation data
    train_data = [
        {"lang": "EN", "model_id": "togethercomputer/Pythia-Chat-Base-7B",
         "model_input": "What did Petra van Staveren win a gold medal for?",
         "model_output_text": "Petra van Staveren won a silver medal in the men's 10 km walk at the 2008 Summer Olympics in Beijing."}
    ]

    val_data = [
        {"id": "val-en-1", "lang": "EN",
         "model_input": "What did Petra van Staveren win a gold medal for?",
         "model_output_text": "Petra van Stoveren won a silver medal in the 2008 Summer Olympics in Beijing, China.",
         "hard_labels": [[25, 31], [45, 49], [69, 83]]}
    ]

    # Convert to DataFrame
    train_df = pd.DataFrame(train_data)
    val_df = pd.DataFrame(val_data)

    # Step 1: Generate a Wikipedia-based summary with self-checking for the model input
    train_df["generated_summary"] = train_df["model_input"].apply(generate_summary_with_self_check)

    # Step 2: Generate a corrected output for the model output
    train_df["corrected_output"] = train_df.apply(
        lambda x: generate_corrected_output(x["generated_summary"], x["model_output_text"]), axis=1
    )

train_df.head()
print(train_df["corrected_output"][0])

Sending request in "generate_summary_with_self_check"
Sending request in "generate_summary_with_self_check"
Sending request in "generate_summary_with_self_check"
Sending request in "self_check_responses"
Sending request in "generate_corrected_output"
Error generating corrected output: Connection error.
